# **<center><font style="color:rgb(100,109,254)">Module 2: Real-Time Controllable Face Makeup</font> </center>**

<img src='https://drive.google.com/uc?export=download&id=1ERLn4WIEsSGnAO_4LDT8R8FTf6ac3f5n'>

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to Face Landmark Detection Theory*

- *Lesson 2: Create a Face Landmarks Detector*

- *Lesson 3: Build a Face Part Selector*

- *Lesson 4: Build a Virtual Face Makeup Application*

- ***Lesson 5:* Build the Final Application** *(This Tutorial)*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [1]:
import cv2
import itertools
import numpy as np
import mediapipe as mp
from collections import deque
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from previous_lesson import (detectFacialLandmarks, detectHandsLandmarks,
                             recognizeGestures, calculateDistance,
                             selectFacePart, getFacePartMask, applyMakeup)
from importlib.metadata import version
print(f"Mediapipe version: {version('mediapipe')}, it should be 0.8.9.1")

Mediapipe version: 0.8.10.1, it should be 0.8.9.1


## **<font style="color:rgb(134,19,348)">Initialize the Face and Hands Landmarks Detection Models</font>**

After that, we will initialize the **`mp.solutions.face_mesh`**, and **`mp.solutions.hands`** classes and then set up the **`mp.solutions.face_mesh.FaceMesh()`**, and **`mp.solutions.hands.Hands()`** functions with appropriate arguments as we have been doing in the previous lessons.

In [2]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the face mesh function with appropriate arguments.
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True,
                                  min_detection_confidence=0.8, min_tracking_confidence=0.5)

# Set up the Hands function with appropriate arguments.
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2,
                       min_detection_confidence=0.8, min_tracking_confidence=0.8)

## **<font style="color:rgb(134,19,348)">Build the Final Application</font>**

Now finally, we will put everything we learned and created in the previous lessons, together to build the final application. We will apply virtual face makeup and modify its intensity in real-time utilizing our hand gestures.

In [3]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Create named window for resizing purposes.
cv2.namedWindow('Real-Time Controllable Face Makeup', cv2.WINDOW_NORMAL)

# Initialize the minimum hue, saturation, and brightness scale factors. 
min_saturation= min_brightness = 1
min_hue = -100

# Initialize the maximum hue, saturation, and brightness scale factors. 
max_saturation= max_brightness = 2
max_hue = 100

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=20)

# Create a buffer to store the selected face part.
selected_face_part = deque([], maxlen=20)

# Initialize the face hue, saturation, and value channel scale factors.
face_hsv_value = [1,1.3,1.1]

# Initialize the lips hue, saturation, and value channel scale factors.
lips_hsv_value = [0,1.6,1.1]

# Initialize the eyes hue, saturation, and value channel scale factors.
eyes_hsv_value = [1,1,0]

# Initialize the eyes hue, saturation, and value channel scale factors.
eyebrows_hsv_value = [0,1.6,1.1]

# Initialize a variable to store the current mode.
current_mode = None

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Initialize variables to store hue, saturaton, and brightness scale factors.
    hue = saturation = brightness = None
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Face landmarks detection.
    frame, face_landmarks = detectFacialLandmarks(frame, face_mesh, draw=False, display=False)
    
    # Check if the Face landmarks in the frame are detected.
    if len(face_landmarks) > 0:
        
        # Perform Hands landmarks detection on the frame.
        _, hands_results = detectHandsLandmarks(frame, hands, draw=False, display=False)
        
         # Check if the hands landmarks in the frame are detected.
        if hands_results.multi_hand_landmarks:
            
            # Perform the Face part selection process utilizing the face and hands landmarks.
            frame, currently_selected_part = selectFacePart(frame, face_landmarks, hands_results)
            
            # Check if a face part is selected.
            if currently_selected_part:
                
                # Append the selected Face part into the buffer.
                selected_face_part.append(currently_selected_part)
                    
            # Check if the length of the buffer is equal to 20 i.e. the max length.
            if len(selected_face_part) == 20:

                # Perform left hand gesture recognition.
                current_gesture, size_gesture_tip_pts = recognizeGestures(frame, hands_results,
                                                                          hand_label='LEFT',
                                                                          draw=False, display=False)

                # Check if a known gesture is recognized.
                if current_gesture != 'UNKNOWN':

                    # Check if all the gestures stored in the buffer are equal to the current gesture.
                    if all(current_gesture==gesture for gesture in buffer):

                        # Append the current gesture into the buffer.
                        buffer.append(current_gesture)

                    # Otherwise.
                    else:

                        # Clear the buffer.
                        buffer.clear()

                    # Check if the length of the buffer is equal to 20 i.e. the max length.
                    if len(buffer) == 5:

                        # Calculate the distance between the middle finger tip and thumb tip landmark of the right hand.
                        distance = calculateDistance(frame, size_gesture_tip_pts['MIDDLE'],
                                                     size_gesture_tip_pts['THUMB'], display=False)


                        # Check if the distance is calculated successfully.
                        # This will be none in case when the hand is not in the frame.
                        if distance:

                            # Check if the current hand gesture is INDEX POINTING UP.
                            if current_gesture == 'INDEX POINTING UP':
                                
                                # Get the interpolation function.
                                hue_interp_f = interp1d([30,230], [min_hue, max_hue])
                                
                                # Calculate the hue scale factor based on the calculated distance.
                                # Higher the distance, higher the hue scale factor will be.
                                hue = hue_interp_f(distance)
                                
                                # Update the current mode value to Hue.
                                current_mode = 'Hue'

                            # Check if the current hand gesture is VICTORY.
                            elif current_gesture == 'VICTORY':
                                
                                # Get the interpolation function and calculate the saturation scale factor.
                                saturation_interp_f = interp1d([30,230], [min_saturation, max_saturation])
                                saturation = saturation_interp_f(distance)
                                
                                # Update the current mode value to Saturation.
                                current_mode = 'Saturation'

                            # Check if the current hand gesture is SPIDERMAN.
                            elif current_gesture == 'SPIDERMAN':
                                
                                # Get the interpolation function and calculate the brightness scale factor.
                                brightness_interp_f = interp1d([30,230], [min_brightness, max_brightness])
                                brightness = brightness_interp_f(distance)
                                
                                # Update the current mode value to Brightness.
                                current_mode = 'Brightness'
                            
                            if current_mode:
                                
                                # Get the interpolation function and calculate the bar value.
                                # This will be used to draw a filled rectangle of height varying with the distance.
                                bar_interp_f = interp1d([30,230],  [frame_height-50, frame_height-400])
                                bar_value = bar_interp_f(distance)  

                                # Write the current mode on the frame.
                                cv2.putText(frame, f'{current_mode} {(distance-30)//2}%',
                                            (frame_width//2-(20*len(current_mode)), 40),
                                            cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)

                                # Draw the filled rectangle with varying height on the frame.
                                cv2.rectangle(frame, (frame_width-80, int(bar_value)), 
                                              (frame_width-50, frame_height-50), (255, 0, 255), -1)

                                # Draw another rectangle around the filled rectangle on the frame.
                                cv2.rectangle(frame, (frame_width-80, frame_height-400),
                                              (frame_width-50, frame_height-50), (0, 255, 0), 6)
                        
                        # Check if the current hand gesture is HIGH-FIVE.
                        if current_gesture == 'HIGH-FIVE':
                            
                            # Update the hsv scale factors of face, lips, and eyes to zero.
                            # This will remove all the makeup from the face.
                            face_hsv_value = [0,1,1]
                            lips_hsv_value = [0,1,1]
                            eyes_hsv_value = [0,1,1]
                            
                            # Clear the selected face part buffer.
                            selected_face_part.clear()
                
                # Check if the length of the buffer is equal to 20 i.e. the max length.
                if len(selected_face_part) == 20:
                    
                    # Check if the current maximum face part selection results in the buffer are LIPS.
                    if max(set(selected_face_part), key=selected_face_part.count) == 'LIPS':

                        # Write LIPS Selected text on the frame. 
                        cv2.putText(frame, 'LIPS Selected.', (5, int(frame_height-20)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                        
                        # Check if a valid hue scale factor value is calculated.
                        if hue:
                            # Update the hue scale factor in the lips hsv scale factors list.
                            lips_hsv_value[0] = hue
                        
                        # Check if a valid saturation scale factor value is calculated.
                        if saturation:
                            # Update the saturation scale factor in the lips hsv scale factors list.
                            lips_hsv_value[1] = saturation
                        
                        # Check if a valid brightness scale factor value is calculated.
                        if brightness:
                            # Update the brightness scale factor in the lips hsv scale factors list.
                            lips_hsv_value[2] = brightness


                    # Check if the current maximum face part selection results in the buffer are FACE.
                    elif max(set(selected_face_part), key=selected_face_part.count) == 'FACE':

                        # Write FACE Selected text on the frame. 
                        cv2.putText(frame, 'FACE Selected.', (5, int(frame_height-20)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

                        # Check if a valid hue scale factor value is calculated.
                        if hue:
                            # Update the hue scale factor in the face hsv scale factors list.
                            face_hsv_value[0] = hue
                        
                        # Check if a valid saturation scale factor value is calculated.
                        if saturation:
                            # Update the saturation scale factor in the face hsv scale factors list.
                            face_hsv_value[1] = saturation
                        
                        # Check if a valid brightness scale factor value is calculated.
                        if brightness:
                            # Update the brightness scale factor in the face hsv scale factors list.
                            face_hsv_value[2] = brightness

                    # Check if the current maximum face part selection results in the buffer are EYES.
                    elif max(set(selected_face_part), key=selected_face_part.count) == 'EYES':

                        # Write EYES Selected text on the frame. 
                        cv2.putText(frame, 'EYES Selected.', (5, int(frame_height-20)),
                                     cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                        
                        # Check if a valid hue scale factor value is calculated.
                        if hue:
                            # Update the hue scale factor in the eyes hsv scale factors list.
                            eyes_hsv_value[0] = hue
                            
                        # Check if a valid saturation scale factor value is calculated.
                        if saturation:
                            # Update the saturation scale factor in the eyes hsv scale factors list.
                            eyes_hsv_value[1] = saturation

                        # Check if a valid brightness scale factor value is calculated.
                        if brightness:
                            # Update the brightness scale factor in the eyes hsv scale factors list.
                            eyes_hsv_value[2] = brightness
                    # Check if the current maximum face part selection results in the buffer are EYES.
                    elif max(set(selected_face_part), key=selected_face_part.count) == 'RIGHT EYEBROW' or max(set(selected_face_part), key=selected_face_part.count) == 'LEFT EYEBROW':

                        # Write EYES Selected text on the frame. 
                        cv2.putText(frame, 'EYEBROWS Selected.', (5, int(frame_height-20)),
                                     cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                        
                        # Check if a valid hue scale factor value is calculated.
                        if hue:
                            # Update the hue scale factor in the eyes hsv scale factors list.
                            eyebrows_hsv_value[0] = hue
                            
                        # Check if a valid saturation scale factor value is calculated.
                        if saturation:
                            # Update the saturation scale factor in the eyes hsv scale factors list.
                            eyebrows_hsv_value[1] = saturation

                        # Check if a valid brightness scale factor value is calculated.
                        if brightness:
                            # Update the brightness scale factor in the eyes hsv scale factors list.
                            eyebrows_hsv_value[2] = brightness
                            
        # Otherwise.
        else:
            # Clear the buffer.
            buffer.clear()
            
        # Extract the foundation mask of the face in the frame.
        face_mask = getFacePartMask(frame, face_landmarks, face_part='FACE', display=False)
        
        # Extract the lipstick mask of the face in the frame.
        lips_mask = getFacePartMask(frame, face_landmarks, face_part='LIPS', display=False)
        
        # Extract the eyeliner mask of the face in the frame.
        eyes_mask = getFacePartMask(frame, face_landmarks, face_part='EYES', display=False)
        
        # Extract the eyeliner mask of the face in the frame.
        eyebrows_mask = getFacePartMask(frame, face_landmarks, face_part='EYEBROWS', display=False)
        
        # Apply the virtual foundation makeup on the face in the frame.
        frame = applyMakeup(frame, face_mask, face_hsv_value, display=False)
        
        # Apply the virtual lipstick on the face in the frame.
        frame = applyMakeup(frame, lips_mask, lips_hsv_value, display=False)
        
        # Apply the virtual eyeliner on the face in the frame.
        frame = applyMakeup(frame, eyes_mask, eyes_hsv_value, display=False)
        
        # Apply the virtual eyeliner on the face in the frame.
        frame = applyMakeup(frame, eyebrows_mask, eyebrows_hsv_value, display=False)
    # Display the frame.
    cv2.imshow('Real-Time Controllable Face Makeup', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()




# Additional Comments:
#       - In the final output I added in the eyebrow make up function.
#         I have tested it, and it does work. However, because of my
#         laptop's low specification, there was a lot of lag.

Smooth! all the components are successfully integrated. 

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```
